# History Compaction

Over many turns, conversation history accumulates. `HistoryCompactor` summarizes older exchanges when approaching context limits.

In [1]:
from agentic_patterns.core.agents import get_agent, run_agent
from agentic_patterns.core.context.history import HistoryCompactor, CompactionConfig

In [2]:
# Configure with low thresholds for demonstration
config = CompactionConfig(max_tokens=500, target_tokens=200)
compactor = HistoryCompactor(config=config)

print(f"Compaction triggers at: {config.max_tokens} tokens")
print(f"Targets reduction to: {config.target_tokens} tokens")

Compaction triggers at: 500 tokens
Targets reduction to: 200 tokens


In [3]:
def show_messages(messages: list, indent: int = 4):
    """Display messages with content preview."""
    prefix = " " * indent
    if not messages:
        print(f"{prefix}(empty)")
        return
    for i, msg in enumerate(messages):
        parts_info = []
        for part in msg.parts:
            part_type = type(part).__name__
            if hasattr(part, 'content'):
                text = str(part.content).replace('\n', ' ')[:50]
                parts_info.append(f"{part_type}({text}...)")
            else:
                parts_info.append(part_type)
        print(f"{prefix}[{i}] {type(msg).__name__}: {', '.join(parts_info)}")

In [4]:
# Capture what was actually sent to the agent
sent_to_agent = None
compaction_result = None

async def capturing_processor(messages):
    """Processor that captures the compacted history sent to agent."""
    global sent_to_agent, compaction_result
    original_tokens = compactor.count_tokens(messages)
    compacted = await compactor.compact(messages)
    compacted_tokens = compactor.count_tokens(compacted)
    
    sent_to_agent = compacted
    if len(compacted) != len(messages):
        compaction_result = {
            "original_msgs": len(messages),
            "compacted_msgs": len(compacted),
            "original_tokens": original_tokens,
            "compacted_tokens": compacted_tokens,
        }
    else:
        compaction_result = None
    return compacted

In [5]:
agent_with_compaction = get_agent(
    system_prompt="You are a helpful assistant.",
    history_processors=[capturing_processor]
)

In [6]:
prompts = [
    "Explain what microservices architecture is.",
    "What are the main benefits?",
    "What are common challenges?",
    "How does it compare to monolithic architecture?",
]

message_history = None

for i, prompt in enumerate(prompts, 1):
    print(f"\n{'='*70}")
    print(f"TURN {i}: {prompt}")
    print(f"{'='*70}")
    
    result, _ = await run_agent(agent_with_compaction, prompt, message_history=message_history)
    
    # 1. SENT TO AGENT (what was actually sent, after compaction)
    if compaction_result:
        print(f"\n  *** COMPACTION: {compaction_result['original_msgs']} msgs ({compaction_result['original_tokens']} tokens) "
              f"-> {compaction_result['compacted_msgs']} msgs ({compaction_result['compacted_tokens']} tokens) ***")
    
    print(f"\n  SENT TO AGENT ({len(sent_to_agent)} messages, {compactor.count_tokens(sent_to_agent)} tokens):")
    show_messages(sent_to_agent)
    
    # 2. NEW MESSAGES (response from this turn)
    new_msgs = list(result.new_messages())
    print(f"\n  NEW MESSAGES ({len(new_msgs)} messages, {compactor.count_tokens(new_msgs)} tokens):")
    show_messages(new_msgs)
    
    # 3. Accumulate to uncompressed history
    if message_history is None:
        message_history = new_msgs
    else:
        message_history = message_history + new_msgs
    
    print(f"\n  UNCOMPRESSED HISTORY ({len(message_history)} messages, {compactor.count_tokens(message_history)} tokens):")
    show_messages(message_history)


TURN 1: Explain what microservices architecture is.

  SENT TO AGENT (1 messages, 94 tokens):
    [0] ModelRequest: SystemPromptPart(You are a helpful assistant....), UserPromptPart(Explain what microservices architecture is....)

  NEW MESSAGES (2 messages, 467 tokens):
    [0] ModelRequest: SystemPromptPart(You are a helpful assistant....), UserPromptPart(Explain what microservices architecture is....)
    [1] ModelResponse: TextPart(# Microservices Architecture  **Microservices arch...)

  UNCOMPRESSED HISTORY (2 messages, 467 tokens):
    [0] ModelRequest: SystemPromptPart(You are a helpful assistant....), UserPromptPart(Explain what microservices architecture is....)
    [1] ModelResponse: TextPart(# Microservices Architecture  **Microservices arch...)

TURN 2: What are the main benefits?

  *** COMPACTION: 3 msgs (510 tokens) -> 2 msgs (267 tokens) ***

  SENT TO AGENT (2 messages, 267 tokens):
    [0] ModelRequest: UserPromptPart(This session is being continued from a previous 